*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [ ]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from wordcloud import WordCloud
import nltk
from nltk import word_tokenize
nltk.download('punkt')
import string
from nltk.corpus import stopwords
nltk.download('stopwords')
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.svm import SVC
from xgboost import XGBRegressor

print(tf.__version__)

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

In [ ]:
df_train = pd.read_csv(train_file_path, sep='\t', header=None, names=['label', 'message'])
df_test = pd.read_csv(test_file_path, sep='\t', header=None, names=['label', 'message'])
df_test.head()

In [ ]:
# Convert labels to categorical data and replace text with code values

df_train['label'] = df_train['label'].astype('category').cat.codes
df_test['label'] = df_test['label'].astype('category').cat.codes

In [ ]:
# Split data into labels and features

train_features = df_train.copy()
test_features = df_test.copy()
train_labels = train_features.pop('label')
test_labels = test_features.pop('label')

In [ ]:
df_train['label'].value_counts().plot.bar();

In [ ]:
df_len = pd.DataFrame(df_train)
df_len['length'] = df_len['message'].apply(len)
df_len.hist(column='length', by='label', bins=range(0, 1000, 10), figsize=(12,4));

In [ ]:
ham_words = ' '.join(list(df_train[df_train['label'] == 0]['message']))
ham_wc = WordCloud(width=512, height=512).generate(ham_words)
plt.figure(figsize=(10,6), facecolor='k')
plt.imshow(ham_wc)
plt.axis('off')
plt.tight_layout(pad=0)
plt.show()

In [ ]:
# Define function for message processing
def process_message(msg):
  # Make everything lowercase and skip punctuation characters
  msg = ''.join(char for char in msg.lower() if char not in set(string.punctuation))
  # Tokenize message but skip stopwords like 'the', 'a', etc.
  word_list = [word for word in word_tokenize(msg) if word not in stopwords.words('english')]
  return word_list

In [ ]:
# Create bag-of-words transformer
# Concatenate train and test data
data_features = pd.concat([train_features, test_features])
data_labels = pd.concat([train_labels, test_labels])
# Create bag-of-words transformer that keeps vocabulary with all words
bow_transformer = CountVectorizer(analyzer=process_message).fit(data_features['message'])
# Print number of words in vocabulary
print(len(bow_transformer.vocabulary_))

In [ ]:
sample_msg = train_features['message'][0]
bow_sample_msg = bow_transformer.transform([sample_msg])
print(sample_msg)
print(bow_sample_msg)

In [ ]:
bow_data = bow_transformer.transform(data_features['message'])
print(bow_data.shape)

In [ ]:
print(bow_data.nnz / (bow_data.shape[0] * bow_data.shape[1]) * 100)

In [ ]:
tfidf_transformer = TfidfTransformer().fit(bow_data)

In [ ]:
tfidf_sample = tfidf_transformer.transform(bow_sample_msg)
print(tfidf_sample)

In [ ]:
data_tfidf = tfidf_transformer.transform(bow_data)
np.shape(data_tfidf)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_tfidf, data_labels, test_size=0.25, random_state=0)

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=250, random_state=0)
rf_clf.fit(X_train, y_train)
y_pred = rf_clf.predict(X_test)

In [ ]:
xgb_clf = XGBRegressor(n_estimators=120, learning_rate=0.125)
xgb_clf.fit(X_train, y_train)
y_pred = xgb_clf.predict(X_test)

In [ ]:
# Third attempt: use GridSearchCV classifier. This method is the best according to the classification results compared to the other methods that were tested.
param_grid = {'C': [0.1, 1, 10, 100],
  'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
  'gamma':['scale', 'auto'],
  'kernel': ['linear']}
   
gsc_clf = GridSearchCV(SVC(probability=True), param_grid, refit=True, verbose=0, n_jobs=-1)
gsc_clf.fit(X_train, y_train)
y_pred = gsc_clf.predict(X_test)

In [ ]:
print(confusion_matrix(y_test,(y_pred>0.5)))
print(classification_report(y_test,(y_pred>0.5)))
print(accuracy_score(y_test,(y_pred>0.5)))

In [ ]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  bow_pred_text = bow_transformer.transform([pred_text])
  tfidf_pred_text = tfidf_transformer.transform(bow_pred_text)
  pred = gsc_clf.predict_proba(tfidf_pred_text)[0]
  prediction = [pred[1], 'ham' if pred[0] > 0.5 else 'spam']


  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

In [ ]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()
